In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Proyecto final Bioinformática (NLP)

EDX: https://dl-itba.croma.ai/courses/course-v1:itba-deeplearning+2020Q2+2021-T1/coursewar[…]2Btype%40vertical%2Bblock%40ac416b55402c48acb7d21c09b2480c54


GITHUB: https://github.com/deeplearning-itba/bioinformatics_final_project

Mirar el siguiente video. Sobre todo la primer parte como para tener contexto sobre descubrimiento de drogas:

https://www.youtube.com/watch?v=jBlTQjcKuaY

GITHUB:
- Parts 1-5  https://github.com/dataprofessor/bioinformatics_freecodecamp/
- Part 6 https://github.com/dataprofessor/bioactivity-prediction-app

Blog Medium:
-https://data-professor.medium.com/

Resumen y puntos importantes del video:
- Queremos entender la bioactividad de una molecúla (molecule_chembl_id) sobre una encima (Acetylcholinesterase)
- La bioactividad se medirá en este caso con el IC50 (standard_value)
- A menor IC50, menos droga para generar la misma actividad, es decir, mayor actividad relativa
- La notación de la fórmula química se llama smiles (https://en.wikipedia.org/wiki/Simplified_molecular-input_line-entry_system)
- Existen distintas técnicas para obtener features de las moléculas y en el video se describen 2:
    - Descriptores de Lipinski
    - Fingerprints del tipo pubchem
- Se construye un modelo de regresión con RandomForest para estimar el pI50 (IC50 en escala logarítimica) dado los fingerprints de entrada

### Objetivos del proyecto:
- Evaluar distintas alternativas de modelos de deep learning para resolver este problema
    - LSTM
    - CNN
    - TextCNN
- Mejorar la métrica del RandomForest
- En vez de ingresar con los features de entrada (fingerprints) como en el video, utilizar técnicas de embeddings usuales en NLP
    - Tokenización en modo caracter dado el smiles de la fórmula química
    - Utilizando un tokenizer sobre los smiles
    - Puedo usar técnicas modernas de tokenización (https://deepchem.readthedocs.io/en/2.4.0/api_reference/tokenizers.html)
- La salida a estimar por el modelo será el pIC50
- La métrica, para comparar con los resultados del RandomForest será el $R^2$

### Librerías:
- chembl-webresource-client: Para bajar el dataset (https://pypi.org/project/chembl-webresource-client/)
- deepchem: libería muy interesante con muchas implementaciones de deep learning aplicadas a la química (https://github.com/deepchem/deepchem)

In [ ]:
import pandas as pd
from chembl_webresource_client.new_client import new_client

# Cliente API

En esta notebook solo se baja el dataset. No tiene que hacer nada más que ejecutarla y entenderla

Librería para baja el dataset

In [ ]:
target = new_client.target
target_query = target.search('acetylcholinesterase')
targets = pd.DataFrame.from_dict(target_query)

In [ ]:
targets.head()

,cross_references,organism,pref_name,score,species_group_flag,target_chembl_id,target_components,target_type,tax_id
0,"[{'xref_id': 'P22303', 'xref_name': None, 'xre...",Homo sapiens,Acetylcholinesterase,27.0,False,CHEMBL220,"[{'accession': 'P22303', 'component_descriptio...",SINGLE PROTEIN,9606
1,[],Homo sapiens,Cholinesterases; ACHE & BCHE,27.0,False,CHEMBL2095233,"[{'accession': 'P06276', 'component_descriptio...",SELECTIVITY GROUP,9606
2,[],Drosophila melanogaster,Acetylcholinesterase,17.0,False,CHEMBL2242744,"[{'accession': 'P07140', 'component_descriptio...",SINGLE PROTEIN,7227
3,"[{'xref_id': 'P04058', 'xref_name': None, 'xre...",Torpedo californica,Acetylcholinesterase,15.0,False,CHEMBL4780,"[{'accession': 'P04058', 'component_descriptio...",SINGLE PROTEIN,7787
4,"[{'xref_id': 'P21836', 'xref_name': None, 'xre...",Mus musculus,Acetylcholinesterase,15.0,False,CHEMBL3198,"[{'accession': 'P21836', 'component_descriptio...",SINGLE PROTEIN,10090


In [ ]:
#CHEMBL220 = Acetilcolinesterasa es un ensima humana. https://es.wikipedia.org/wiki/Acetilcolinesterasa
#Modular la actividad de la acetilcolinesterasa es una de las maniobras terapéuticas fundamentadas en aliviar las funciones cognitivas en pacientes con la enfermedad de Alzheimer.5​

selected_target = targets.target_chembl_id[0]
selected_target
# 'CHEMBL220'

'CHEMBL220'

In [ ]:
# Here, we will retrieve only bioactivity data for Human Acetylcholinesterase (CHEMBL220) that are reported as pChEMBL values.
activity = new_client.activity
res = activity.filter(target_chembl_id=selected_target).filter(standard_type="IC50")

# Bajada de data
Puede tardar un poco dependiendo de que tan saturado este el server

Por eso el for, para ver el progreso y bajar la ansiedad. Son en el orden de 7500K

In [ ]:
res_cols = []
for i, r in enumerate(res):
    print(f'{i}\r', end='')
    res_cols.append(r)

In [ ]:
df = pd.DataFrame(res_cols)

In [ ]:
df['canonical_smiles']

0                   CCOc1nn(-c2cccc(OCc3ccccc3)c2)c(=O)o1
1              O=C(N1CCCCC1)n1nc(-c2ccc(Cl)cc2)nc1SCC1CC1
2       CN(C(=O)n1nc(-c2ccc(Cl)cc2)nc1SCC(F)(F)F)c1ccccc1
3           O=C(N1CCCCC1)n1nc(-c2ccc(Cl)cc2)nc1SCC(F)(F)F
4               CSc1nc(-c2ccc(OC(F)(F)F)cc2)nn1C(=O)N(C)C
                              ...                        
7544     COc1ccc(CCC(=O)Nc2nc(-c3cc4ccccc4oc3=O)cs2)cc1OC
7545                COc1ccc(-c2csc(NC(=O)CCN3CCCC3)n2)cc1
7546    COc1cc(C2C3=C(CCCC3=O)NC3=C2C(=O)CCC3)ccc1OCc1...
7547    O=C1CCCC2=C1C(c1ccc(OCc3cccc(F)c3)c(Br)c1)C1=C...
7548                              Nc1c2c(nc3ccccc13)CCCC2
Name: canonical_smiles, Length: 7549, dtype: object

# Limpio data

In [ ]:
#standard_value: es la potencia de la droga, cuanta concentracion de la dorga se requiere para obtener una Inhibicion del 50% (IC50)
# IC50: Una medida que nos indica la concentración a la que un fármaco es capaz de inhibir un proceso biológico concreto en un 50%

df = df.dropna(subset=['standard_value', 'canonical_smiles']) 
df = df.drop_duplicates(['canonical_smiles'])  # molecule_chembl_id o canonical_smiles son el id de cada molecula, puede ser que en el dataset se repitan, hacemos un drop porque es nuestro Primary Key
#La notación de la fórmula química se llama smiles (https://en.wikipedia.org/wiki/Simplified_molecular-input_line-entry_system)
#SMILES: describir sin ambigüedades la estructura de una molécula usando cadenas ASCII cortas

In [ ]:
selection = ['molecule_chembl_id','canonical_smiles','standard_value']
df = df[selection]

# Preprocesamiento y normalización

In [ ]:
import numpy as np

In [ ]:
# Normalizacion de standard_value
df['standard_value'] = df['standard_value'].apply(pd.to_numeric)

In [ ]:
# En el video se hace esta normalización. No la veo del todo necesaria
df['standard_value_norm'] = df['standard_value'].apply(lambda x: (x>1e8)*1e8 + (x<=1e8)*x)

In [ ]:
#pI50 (IC50 en escala logarítimica) dado los fingerprints de entrada
df['pIC50'] = df['standard_value'].apply(lambda x: -np.log10(x*(10**-9)))

In [ ]:
! mkdir data

mkdir: cannot create directory ‘data’: File exists


In [ ]:
df.to_csv('data/acetylcholinesterase_02_bioactivity_data_preprocessed.csv', index=False)

In [ ]:
df.head()

,molecule_chembl_id,canonical_smiles,standard_value,standard_value_norm,pIC50
0,CHEMBL133897,CCOc1nn(-c2cccc(OCc3ccccc3)c2)c(=O)o1,750.0,750.0,6.124939
1,CHEMBL336398,O=C(N1CCCCC1)n1nc(-c2ccc(Cl)cc2)nc1SCC1CC1,100.0,100.0,7.000000
2,CHEMBL131588,CN(C(=O)n1nc(-c2ccc(Cl)cc2)nc1SCC(F)(F)F)c1ccccc1,50000.0,50000.0,4.301030
3,CHEMBL130628,O=C(N1CCCCC1)n1nc(-c2ccc(Cl)cc2)nc1SCC(F)(F)F,300.0,300.0,6.522879
4,CHEMBL130478,CSc1nc(-c2ccc(OC(F)(F)F)cc2)nn1C(=O)N(C)C,800.0,800.0,6.096910
